In [59]:
# From PyTorch CNN offical tutorial
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data
import numpy as np
import pandas as pd


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(50, 100, 3)
        self.conv2 = nn.Conv1d(50, 100, 4)
        self.conv3 = nn.Conv1d(50, 100, 5)
        self.pool = nn.MaxPool1d(60)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(300, 16)

    def forward(self, x):
        x1 = self.pool(F.relu(self.conv1(x)))
        x2 = self.pool(F.relu(self.conv2(x)))
        x3 = self.pool(F.relu(self.conv3(x)))
        x = torch.cat((x1, x2, x3), dim = 1).squeeze()
        x = self.dropout(x)
        x = self.fc1(x)
        return x


net = Net().to(device)

cuda:0


In [62]:
class Net_dynamic(nn.Module):
    def __init__(self):
        super(Net_dynamic, self).__init__()
        self.emb = nn.Linear(50, 50)
        self.conv1 = nn.Conv1d(50, 100, 3)
        self.conv2 = nn.Conv1d(50, 100, 4)
        self.conv3 = nn.Conv1d(50, 100, 5)
        self.pool = nn.MaxPool1d(60)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(300, 16)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.emb(x)
        x = x.transpose(1, 2)
        x1 = self.pool(F.relu(self.conv1(x)))
        x2 = self.pool(F.relu(self.conv2(x)))
        x3 = self.pool(F.relu(self.conv3(x)))
        x = torch.cat((x1, x2, x3), dim = 1).squeeze()
        x = self.dropout(x)
        x = self.fc1(x)
        return x

net_2 = Net_dynamic().to(device)

In [63]:
class Net_dual(nn.Module):
    def __init__(self):
        super(Net_dual, self).__init__()
        self.emb = nn.Linear(50, 50)
        self.conv1 = nn.Conv1d(100, 100, 3)
        self.conv2 = nn.Conv1d(100, 100, 4)
        self.conv3 = nn.Conv1d(100, 100, 5)
        self.pool = nn.MaxPool1d(60)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(300, 16)

    def forward(self, x):
        x_emb = self.emb(x.transpose(1, 2)).transpose(1, 2)
        x = torch.cat((x, x_emb), dim = 1)
        x1 = self.pool(F.relu(self.conv1(x)))
        x2 = self.pool(F.relu(self.conv2(x)))
        x3 = self.pool(F.relu(self.conv3(x)))
        x = torch.cat((x1, x2, x3), dim = 1).squeeze()
        x = self.dropout(x)
        x = self.fc1(x)
        return x

net_3 = Net_dual().to(device)

In [13]:
label_train = list(set(list(np.load('D:/Courses/11747/label_train.npy'))))
label_dict = dict()
for idx, key in enumerate(label_train):
    label_dict[key] = idx

# add hard coding for mislabel
label_dict['Media and darama'] = label_dict['Media and drama']
print (label_dict)

label_list = list(label_dict.keys())
print (label_list)


{'Agriculture, food and drink': 0, 'Video games': 1, 'Language and literature': 2, 'Philosophy and religion': 3, 'Social sciences and society': 4, 'Natural sciences': 5, 'Mathematics': 6, 'Engineering and technology': 7, 'Miscellaneous': 8, 'Art and architecture': 9, 'Warfare': 10, 'Geography and places': 11, 'Music': 12, 'History': 13, 'Media and drama': 14, 'Sports and recreation': 15, 'Media and darama': 14}
['Agriculture, food and drink', 'Video games', 'Language and literature', 'Philosophy and religion', 'Social sciences and society', 'Natural sciences', 'Mathematics', 'Engineering and technology', 'Miscellaneous', 'Art and architecture', 'Warfare', 'Geography and places', 'Music', 'History', 'Media and drama', 'Sports and recreation', 'Media and darama']


In [5]:
# From PyTorch dataloader offical tutorial

class Sentence():
    """sentence classification dataset."""

    def __init__(self, dataset_dir, label_dir):
        # reshaping data from (N, length, channel) to (N, channel, length) 
        self.data = torch.from_numpy(np.load(dataset_dir)).transpose(1, 2).float().to(device)
        
        # reading raw label data, transform to integer value using dict         
        label_list = np.load(label_dir)
        self.labels = torch.zeros(self.data.shape[0]).to(device)
        for i in range(len(self.labels)):
            try:
                self.labels[i] = label_dict[label_list[i]]
            except:
                pass
        
        print (self.data.shape, self.labels.shape)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        data = self.data[idx]
        label = self.labels[idx]

        sample = {'data': data, 'label': label}

        return sample
    


torch.Size([697, 50, 60]) torch.Size([697])


In [ ]:
dataset_train = Sentence('D:/Courses/11747/data_train.npy', 'D:/Courses/11747/label_train.npy')
dataset_val = Sentence('D:/Courses/11747/data_val.npy', 'D:/Courses/11747/label_val.npy')
dataset_test = Sentence('D:/Courses/11747/data_test.npy', 'D:/Courses/11747/label_test.npy')

In [9]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=32,
                                          shuffle=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=32,
                                          shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=32,
                                          shuffle=False, num_workers=0)


In [29]:
def eval(model, model_name):

    model.eval()
    err = 0
    running_loss = 0.0
    all_pred = []

    for i, data in enumerate(val_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['data']
        labels = data['label'].long()

        # forward + backward + optimize
        outputs = model(inputs)
        pred = torch.argmax(outputs, dim = 1)
        all_pred += pred.tolist()
        err += torch.nonzero(labels - pred).size(0)
        loss = criterion(outputs, labels)

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d / %d] loss: %.3f' %
                  (epoch + 1, i + 1, len(trainloader), running_loss / 2000))
            running_loss = 0.0
    
    # calculating accuracy     
    accu = 1 - err/dataset_val.__len__()
    print (len(all_pred))
    print ('Accuracy: ', accu)
    
    # print output label txt file     
    output_label = []
    for pred_i in all_pred:
        output_label.append(label_list[pred_i])
        
    df = pd.DataFrame(output_label)
    df.to_csv(model_name + "_pred.csv", sep=',',index=False, header=False)
    
    return accu


In [44]:
def test(model, model_name):

    model.eval()
    all_pred = []

    for i, data in enumerate(test_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['data']

        # forward + backward + optimize
        outputs = model(inputs)
        pred = torch.argmax(outputs, dim = 1)
        all_pred += pred.tolist()
        
    # print output label txt file     
    output_label = []
    for pred_i in all_pred:
        output_label.append(label_list[pred_i])
        
    df = pd.DataFrame(output_label)
    df.to_csv(model_name + "_pred_test.csv", sep=',',index=False, header=False)
    

In [65]:
import torch.optim as optim

def train(net, model_name, epochs = 10):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    eval_accu = []
    train_loss = []
    eval(net, model_name)

    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['data']
            labels = data['label'].long()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                train_loss.append(running_loss / 2000)
                print('[%d, %5d / %d] loss: %.3f' %
                      (epoch + 1, i + 1, len(trainloader), running_loss / 2000))
                running_loss = 0.0

        eval_accu.append(eval(net, model_name))
        if eval_accu[-1] == max(eval_accu):
            torch.save(net.state_dict(), model_name + '.best')
    
    df = pd.DataFrame(eval_accu)
    df.to_csv(model_name + '_' + str(epochs) + "_accu.csv", sep=',',index=False, header=False)

    df = pd.DataFrame(train_loss)
    df.to_csv(model_name + '_' + str(epochs) + "_train_loss.csv", sep=',',index=False, header=False)

    print('Finished Training')
    print('Accuracy: ', eval_accu)

In [60]:
# static training: 0.7542768273716952, 0.7822706065318819, 0.8087091757387247, 0.8087091757387247, 0.8009331259720063, 0.8040435458786936, 0.807153965785381, 0.7993779160186625, 0.80248833592535, 0.8040435458786936
# 

train(net, 'static', 50)

643
Accuracy:  0.11819595645412129
[1,  2000 / 7935] loss: 1.834
[1,  4000 / 7935] loss: 1.226
[1,  6000 / 7935] loss: 1.102
643
Accuracy:  0.7651632970451011
[2,  2000 / 7935] loss: 1.034
[2,  4000 / 7935] loss: 1.001
[2,  6000 / 7935] loss: 0.976
643
Accuracy:  0.7931570762052877
[3,  2000 / 7935] loss: 0.965
[3,  4000 / 7935] loss: 0.944
[3,  6000 / 7935] loss: 0.927
643
Accuracy:  0.807153965785381
[4,  2000 / 7935] loss: 0.927
[4,  4000 / 7935] loss: 0.909
[4,  6000 / 7935] loss: 0.895
643
Accuracy:  0.8087091757387247
[5,  2000 / 7935] loss: 0.900
[5,  4000 / 7935] loss: 0.884
[5,  6000 / 7935] loss: 0.871
643
Accuracy:  0.8133748055987559
[6,  2000 / 7935] loss: 0.879
[6,  4000 / 7935] loss: 0.864
[6,  6000 / 7935] loss: 0.852
643
Accuracy:  0.8195956454121307
[7,  2000 / 7935] loss: 0.862
[7,  4000 / 7935] loss: 0.846
[7,  6000 / 7935] loss: 0.836
643
Accuracy:  0.8242612752721618
[8,  2000 / 7935] loss: 0.847
[8,  4000 / 7935] loss: 0.831
[8,  6000 / 7935] loss: 0.821
643
Accu

In [66]:
train(net_2, 'dynamic', 50)

643
Accuracy:  0.055987558320373276
[1,  2000 / 7935] loss: 1.913
[1,  4000 / 7935] loss: 1.251
[1,  6000 / 7935] loss: 1.106
643
Accuracy:  0.7589424572317263
[2,  2000 / 7935] loss: 1.033
[2,  4000 / 7935] loss: 1.000
[2,  6000 / 7935] loss: 0.976
643
Accuracy:  0.7931570762052877
[3,  2000 / 7935] loss: 0.968
[3,  4000 / 7935] loss: 0.948
[3,  6000 / 7935] loss: 0.932
643
Accuracy:  0.807153965785381
[4,  2000 / 7935] loss: 0.935
[4,  4000 / 7935] loss: 0.918
[4,  6000 / 7935] loss: 0.905
643
Accuracy:  0.8087091757387247
[5,  2000 / 7935] loss: 0.911
[5,  4000 / 7935] loss: 0.895
[5,  6000 / 7935] loss: 0.884
643
Accuracy:  0.8087091757387247
[6,  2000 / 7935] loss: 0.892
[6,  4000 / 7935] loss: 0.876
[6,  6000 / 7935] loss: 0.866
643
Accuracy:  0.807153965785381
[7,  2000 / 7935] loss: 0.874
[7,  4000 / 7935] loss: 0.859
[7,  6000 / 7935] loss: 0.850
643
Accuracy:  0.807153965785381
[8,  2000 / 7935] loss: 0.859
[8,  4000 / 7935] loss: 0.843
[8,  6000 / 7935] loss: 0.835
643
Accur

In [67]:
train(net_3, 'dual', 50)

643
Accuracy:  0.13219284603421466
[1,  2000 / 7935] loss: 1.756
[1,  4000 / 7935] loss: 1.161
[1,  6000 / 7935] loss: 1.060
643
Accuracy:  0.7744945567651633
[2,  2000 / 7935] loss: 1.002
[2,  4000 / 7935] loss: 0.971
[2,  6000 / 7935] loss: 0.948
643
Accuracy:  0.8087091757387247
[3,  2000 / 7935] loss: 0.941
[3,  4000 / 7935] loss: 0.919
[3,  6000 / 7935] loss: 0.903
643
Accuracy:  0.8133748055987559
[4,  2000 / 7935] loss: 0.906
[4,  4000 / 7935] loss: 0.886
[4,  6000 / 7935] loss: 0.873
643
Accuracy:  0.8164852255054432
[5,  2000 / 7935] loss: 0.880
[5,  4000 / 7935] loss: 0.861
[5,  6000 / 7935] loss: 0.850
643
Accuracy:  0.8164852255054432
[6,  2000 / 7935] loss: 0.858
[6,  4000 / 7935] loss: 0.840
[6,  6000 / 7935] loss: 0.830
643
Accuracy:  0.8180404354587869
[7,  2000 / 7935] loss: 0.840
[7,  4000 / 7935] loss: 0.822
[7,  6000 / 7935] loss: 0.812
643
Accuracy:  0.8227060653188181
[8,  2000 / 7935] loss: 0.824
[8,  4000 / 7935] loss: 0.806
[8,  6000 / 7935] loss: 0.797
643
Acc

In [68]:
test(net_3, 'dual')

In [70]:
net_best = Net_dual().to(device)
net_best.load_state_dict(torch.load('dual.best'))
eval(net_best, 'model_best')
test(net_best, 'model_best')

643
Accuracy:  0.8227060653188181
